In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score

In [2]:
data=pd.read_csv("C:\\Users\\agnes\\Documents\\apziva_ai_residency\\project2\\data\\term-deposit-marketing-2020.csv")

In [3]:
data['y'].value_counts()

no     37104
yes     2896
Name: y, dtype: int64

In [4]:
data['job']=data['job'].astype('category')
data['marital']=data['marital'].astype('category')
data['education']=data['education'].astype('category')
data['default']=data['default'].astype('category')
data['housing']=data['housing'].astype('category')
data['loan']=data['loan'].astype('category')
data['contact']=data['contact'].astype('category')
data['month']=data['month'].astype('category')
data['y']=data['y'].astype('category')
cat_columns = data.select_dtypes(['category']).columns
data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)

In [5]:
matrix=data.corr()
print(matrix)

                age       job   marital  education   default   balance  \
age        1.000000 -0.019719 -0.386716  -0.091083 -0.014857  0.081517   
job       -0.019719  1.000000  0.051588   0.174336 -0.004977  0.015187   
marital   -0.386716  0.051588  1.000000   0.095899 -0.004595  0.008542   
education -0.091083  0.174336  0.095899   1.000000 -0.006647  0.066592   
default   -0.014857 -0.004977 -0.004595  -0.006647  1.000000 -0.070069   
balance    0.081517  0.015187  0.008542   0.066592 -0.070069  1.000000   
housing   -0.179606 -0.114148  0.004453  -0.083699 -0.019229 -0.049946   
loan       0.000041 -0.024220 -0.042975  -0.043702  0.075006 -0.080596   
contact    0.041114 -0.077812 -0.029242  -0.096023  0.006881 -0.011681   
day       -0.011689  0.028441  0.000099   0.036206  0.006287  0.011662   
month     -0.063437 -0.106735 -0.006779  -0.066100  0.013660  0.023718   
duration  -0.036270 -0.002445  0.015772   0.001828 -0.008597  0.014072   
campaign   0.016204  0.013914 -0.00125

In [6]:
x= data[['age', 'job','marital','education','balance','housing','day','month','duration','campaign']]
y=data[['y']]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.2, random_state=0)

In [7]:
lr=LogisticRegression()
lr.fit(x_train,y_train.values.ravel())
predictions=lr.predict(x_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.94      0.99      0.96      7425
           1       0.57      0.21      0.31       575

    accuracy                           0.93      8000
   macro avg       0.76      0.60      0.64      8000
weighted avg       0.92      0.93      0.92      8000



C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
from imblearn.under_sampling import RandomUnderSampler  

under_sampler = RandomUnderSampler()
X_res, y_res = under_sampler.fit_resample(x, y)

In [9]:
x_train_res, y_train_res=under_sampler.fit_resample(x_train,y_train)

In [10]:
lr2=LogisticRegression()
lr2.fit(x_train_res, y_train_res.values.ravel())
predictions=lr2.predict(x_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.98      0.84      0.91      7425
           1       0.27      0.76      0.40       575

    accuracy                           0.84      8000
   macro avg       0.63      0.80      0.65      8000
weighted avg       0.93      0.84      0.87      8000



C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
f1_score(y_test,predictions)

0.40110395584176634

In [12]:
## adding grid search 
parameters = [ {'max_depth': [2, 4, 6, 10], 'min_samples_leaf': [2, 3, 4, 5, 7], 'min_samples_split' : [2, 3, 4, 5]}]


In [14]:
#
LR = LogisticRegression()
LRparam_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
     'max_iter': list(range(100,10000,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear']
}
#search = RandomizedSearchCV(model, space, n_iter=100000, scoring='f1', n_jobs=-1, cv=cv, random_state=1)
lr_grid={'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

LR_search = GridSearchCV(LR, param_grid=grid, refit = True, verbose = 3, cv=5,scoring=f1)

#result = search.fit(x, y)


In [16]:
LR_search.fit(x_train_res, y_train_res.values.ravel())


Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END ...............C=0.001, penalty=l2;, score=0.810 total time=   0.0s
[CV 2/5] END ...............C=0.001, penalty=l2;, score=0.804 total time=   0.0s
[CV 3/5] END ...............C=0.001, penalty=l2;, score=0.790 total time=   0.0s
[CV 4/5] END ...............C=0.001, penalty=l2;, score=0.792 total time=   0.0s
[CV 5/5] END ...............C=0.001, penalty=l2;, score=0.814 total time=   0.0s
[CV 1/5] END ..................C=0.01, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ..................C=0.01, penalty=l

C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[CV 2/5] END ................C=0.01, penalty=l2;, score=0.793 total time=   0.0s
[CV 3/5] END ................C=0.01, penalty=l2;, score=0.795 total time=   0.0s
[CV 4/5] END ................C=0.01, penalty=l2;, score=0.798 total time=   0.0s
[CV 5/5] END ................C=0.01, penalty=l2;, score=0.818 total time=   0.0s
[CV 1/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ...................C=0.1, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END .................C=0.1, penalty=l2;, score=0.798 total time=   0.0s
[CV 2/5] END .................C=0.1, penalty=l2;, score=0.807 total time=   0.0s
[CV 3/5] END .................C=0.1, penalty=l2;, score=0.795 total time=   0.0s


C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[CV 4/5] END .................C=0.1, penalty=l2;, score=0.789 total time=   0.0s
[CV 5/5] END .................C=0.1, penalty=l2;, score=0.818 total time=   0.0s
[CV 1/5] END ...................C=1.0, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ...................C=1.0, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ...................C=1.0, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ...................C=1.0, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ...................C=1.0, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END .................C=1.0, penalty=l2;, score=0.798 total time=   0.0s
[CV 2/5] END .................C=1.0, penalty=l2;, score=0.808 total time=   0.0s
[CV 3/5] END .................C=1.0, penalty=l2;, score=0.797 total time=   0.0s
[CV 4/5] END .................C=1.0, penalty=l2;, score=0.796 total time=   0.0s
[CV 5/5] END .................C=1.0, penalty=l2;, score=0.818 total time=   0.0s


...

[CV 1/5] END ..................C=10.0, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ..................C=10.0, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ..................C=10.0, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ..................C=10.0, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ..................C=10.0, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END ................C=10.0, penalty=l2;, score=0.793 total time=   0.0s
[CV 2/5] END ................C=10.0, penalty=l2;, score=0.808 total time=   0.0s
[CV 3/5] END ................C=10.0, penalty=l2;, score=0.797 total time=   0.0s
[CV 4/5] END ................C=10.0, penalty=l2;, score=0.795 total time=   0.0s
[CV 5/5] END ................C=10.0, penalty=l2;, score=0.819 total time=   0.0s
[CV 1/5] END .................C=100.0, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END .................C=100.0, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ...............

C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[CV 1/5] END ...............C=100.0, penalty=l2;, score=0.793 total time=   0.0s
[CV 2/5] END ...............C=100.0, penalty=l2;, score=0.795 total time=   0.0s
[CV 3/5] END ...............C=100.0, penalty=l2;, score=0.793 total time=   0.0s
[CV 4/5] END ...............C=100.0, penalty=l2;, score=0.797 total time=   0.0s
[CV 5/5] END ...............C=100.0, penalty=l2;, score=0.819 total time=   0.0s
[CV 1/5] END ................C=1000.0, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ................C=1000.0, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ................C=1000.0, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ................C=1000.0, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ................C=1000.0, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END ..............C=1000.0, penalty=l2;, score=0.803 total time=   0.0s
[CV 2/5] END ..............C=1000.0, penalty=l2;, score=0.799 total time=   0.0s
[CV 3/5] END ..............C

C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             scoring=make_scorer(f1_score, average=macro), verbose=3)

In [17]:
LR_search.best_params_
# summarize
print('Best F1 score Accuracy: %.3f' % LR_search.best_score_)
print('Config: %s' % LR_search.best_params_)


Best F1 score Accuracy: 0.803
Config: {'C': 1.0, 'penalty': 'l2'}


In [18]:
from sklearn.model_selection import KFold 
k = 5
kf = KFold(n_splits=k, random_state=None)

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
result = cross_val_score(LR_search , x, y, cv = kf)
 
print("Avg accuracy: {}".format(result.mean()))


Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END .................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END ...............C=0.001, penalty=l2;, score=0.687 total time=   0.0s


C:\Users\agnes\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example us

[CV 2/5] END ...............C=0.001, penalty=l2;, score=0.641 total time=   0.0s
[CV 3/5] END ...............C=0.001, penalty=l2;, score=0.622 total time=   0.0s


C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (

[CV 4/5] END ...............C=0.001, penalty=l2;, score=0.608 total time=   0.0s
[CV 5/5] END ...............C=0.001, penalty=l2;, score=0.563 total time=   0.0s
[CV 1/5] END ..................C=0.01, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ..................C=0.01, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ..................C=0.01, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ..................C=0.01, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ..................C=0.01, penalty=l1;, score=nan total time=   0.0s


...

C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (

[CV 3/5] END ..............C=1000.0, penalty=l2;, score=0.638 total time=   0.0s
[CV 4/5] END ..............C=1000.0, penalty=l2;, score=0.588 total time=   0.0s


C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (

[CV 5/5] END ..............C=1000.0, penalty=l2;, score=0.581 total time=   0.0s
Avg accuracy: 0.6516015071690147


C:\Users\agnes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
